## IDeepPhase tutorial

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import numpy as np
import pandas as pd
import keras as K
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Activation,Dropout,Flatten,Dense,Convolution2D,MaxPooling2D,GlobalAveragePooling2D
from keras.applications import ResNet50
from keras.models import Model,Sequential



def greenonly(x):
    return x*[0.,1.,0.]

#Setting training and validation data

sz = 512
batch_size = 64
train_data_dir = 'data_train/'
validation_data_dir = 'data_test/'
train_datagen = ImageDataGenerator(rescale=1./255,
                rotation_range=180,
                horizontal_flip = True)
                                   # ,preprocessing_function=greenonly)
test_datagen = ImageDataGenerator(rescale=1./255)
                                  #,preprocessing_function=greenonly)
train_generator = train_datagen.flow_from_directory(train_data_dir,shuffle = True,
                                                    target_size = (sz,sz),
                                                   batch_size = batch_size,class_mode = 'binary')
validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                       shuffle = False,
                                                       target_size = (sz,sz),
                                                       batch_size = batch_size,
                                                       class_mode = 'binary')

if K.backend.image_dim_ordering() == 'th':
    input_shape = (3, sz, sz)
else:
    input_shape = (sz, sz, 3)


In [ ]:
#Building CNN

model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
# Training the model
%%time
model.fit_generator(train_generator,
                    train_generator.n // batch_size,
                   epochs = 50,
                   workers = 8,
                   validation_data = validation_generator,
                   validation_steps = validation_generator.n // batch_size,class_weight=[0.2,0.8]
                   )


In [ ]:
# Or using pretrained weight
model.load_weights('pretrained.h5')

In [ ]:
# Calculating scores for all images

file=os.listdir('data_exam\\').copy()
img=[]
for j in range(100):
    try:
        target_tensor=[]
        target_filename=[]
        for i in file[j::100]:
            try:
                x = image.load_img('data_exam\\'+i, target_size=(512,512))
                x = image.img_to_array(x)
                x = greenonly(x)
                x = x.reshape((1,) + x.shape)
                x = x/255.
                target_tensor.append(x)
                target_filename.append(i)
            except:pass
        score_img=pd.concat([pd.DataFrame(model.predict(np.vstack(target_tensor))),pd.DataFrame(target_filename,columns=['image'])],axis=1).copy()
        img.append(score_img)
    except:pass

In [ ]:
file=os.listdir('data_exam\\').copy()
img=[]
for i in file:
    x = image.load_img('data_exam\\'+i, target_size=(512,512))
    x = image.img_to_array(x)
    x = greenonly(x)
    x = x.reshape((1,) + x.shape)
    x = x/255.
    img.append([i,model.predict(x)[0][0]])

In [ ]:
# Output
pd.DataFrame(img).to_csv('output.csv')